In [2]:
end_date = "2017-09-04"
start_date = "2017-07-18"
xls_path='/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/Auto Run Device Weekday and Hour/Device_Weekday_Hour_From_'+str(start_date)+'_To_'+str(end_date)+'.xlsx'

from pandas import ExcelWriter
import pandas as pd
import numpy as np
pd.options.display.max_rows = 6
import os
import argparse
import httplib2
import pprint
import time
import datetime
from io import StringIO

from googleapiclient.discovery import build
from oauth2client import GOOGLE_TOKEN_URI
from oauth2client.client import OAuth2Credentials
from googleapiclient.errors import HttpError

Group = pd.read_csv("/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/Auto Run Device Weekday and Hour/Group.csv")
Group['floodlightActivity'] = Group['Floodlight activity']
del Group['Floodlight activity']

def create_credentials():
    """Create Google OAuth2 credentials.

    Args:
        client_id: Client id of a Google Cloud console project.
        client_secret: Client secret of a Google Cloud console project.
        refresh_token: A refresh token authorizing the Google Cloud console project
          to access the DS data of some Google user.
    Returns:
        OAuth2Credentials
    """
    return OAuth2Credentials(access_token=None,
                           client_id='549790627766-qnth4m8qvuimg87pnsp4b82lhte7dk5a.apps.googleusercontent.com',
                           client_secret='Vta4lQLOL49vVYvktkcPGRNb',
                           refresh_token='1/ab7pCGMu3K5AveG0UOUpQ0J08vCp6uM357O8qmoPDMs',
                           token_expiry=None,
                           token_uri="https://accounts.google.com/o/oauth2/token",
                           user_agent=None)

def get_service(credentials):
    """Set up a new DoubleClick Search service.

    Args:
        credentials: An OAuth2Credentials generated with create_credentials, or
        flows in the oatuh2client.client package.
    Returns:
        An authorized Doubleclicksearch serivce.
    """
    # Use the authorize() function of OAuth2Credentials to apply necessary credential
    # headers to all requests.
    http = credentials.authorize(http = httplib2.Http())

    # Construct the service object for the interacting with the DoubleClick Search API.
    service = build('doubleclicksearch', 'v2', http=http)
    return service

def poll_report(service, report_id):
    """Poll the API with the reportId until the report is ready, up to ten times.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
    Returns:
        pd.DataFrame, report file
    """
    for _ in range(10):
        try:
            request = service.reports().get(reportId=report_id)
            json_data = request.execute()
            if json_data['isReportReady']:
                pprint.pprint('The report is ready.')

                # For large reports, DS automatically fragments the report into multiple
                # files. The 'files' property in the JSON object that DS returns contains
                # the list of URLs for file fragment. To download a report, DS needs to
                # know the report ID and the index of a file fragment.
                report = pd.DataFrame()
                for i in range(len(json_data['files'])):
                    pprint.pprint('Downloading fragment ' + str(i) + ' for report ' + report_id)
                    report = report.append(download_files(service, report_id, str(i)), ignore_index = True) # See Download the report.
                return report

            else:
                pprint.pprint('Report is not ready. I will try again.')
                time.sleep(10)
        except HttpError as e:
            error = simplejson.loads(e.content)['error']['errors'][0]

            # See Response Codes
            pprint.pprint('HTTP code %d, reason %s' % (e.resp.status, error['reason']))
            break
        
def download_files(service, report_id, report_fragment):
    """Generate and print sample report.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
        report_fragment: The 0-based index of the file fragment from the files array.
    Returns:
        pd.DataFrame report file
    """
    request = service.reports().getFile(reportId=report_id, reportFragment=report_fragment)
    return pd.read_csv(StringIO(request.execute().decode('utf-8')))

def request_report(service, start_date, end_date, columns):
    """Request sample report and print the report ID that DS returns. See Set Up Your Application.

    Args:
        service: An authorized Doublelcicksearch service.
        columns: list of columns will be in the report
    Returns:
        The report id.
    """
    request = service.reports().request(
        body={
                "reportScope": {
                    "agencyId": "20100000000000932",
                    "advertiserId": "21700000001423776"
                },
                "reportType": "account",
                "columns": [{'columnName': column} for column in columns],   
                "timeRange" : {
                    "startDate" : start_date,
                    "endDate" : end_date
                    },
                
                #"filters": [
                #    {
                #        "column" : { "columnName": "keywordLabels" },
                #        "operator" : "containsElement",
                #        "values" : ["JubaNovTest",]
                #    }
                #],
                
                "downloadFormat": "csv",
                "maxRowsPerFile": 100000000,
                "statisticsCurrency": "agency",
                "verifySingleTimeZone": "false",
                "includeRemovedEntities": "false"
            }
    )
    json_data = request.execute()
    return json_data['id']

In [3]:
# Device
creds = create_credentials()

service = get_service(creds)


print(start_date), print(end_date)
REPORTID_nonHVA = request_report(service, start_date, end_date, 
                                 ['accountType', 'deviceSegment', 'clicks', 'cost', 'impr','dfaRevenue', 'dfaTransactions'])
REPORTID_HVA = request_report(service, start_date, end_date, 
                              ['accountType', 'deviceSegment',
                               'floodlightActivity', 'dfaActions'])
non_hva_device= poll_report(service, REPORTID_nonHVA)
hva_device = poll_report(service, REPORTID_HVA)

2017-07-18
2017-09-04
'Report is not ready. I will try again.'
'The report is ready.'
'Downloading fragment 0 for report AAAnTEU9mrlRJVY3'
'The report is ready.'
'Downloading fragment 0 for report AAAnf4esM8nxi-JW'


In [4]:
Device_1=non_hva_device
Device_2=hva_device

Device_2_Group_New=pd.merge(Device_2,Group,on="floodlightActivity")
len(pd.unique(Device_2_Group_New.floodlightActivity)) #53

Device_2_Group_3HVA=Device_2_Group_New.groupby(['deviceSegment','accountType','Group Name'])[['dfaActions']].sum() 
type(Device_2_Group_3HVA)
Device_2_Group_3HVA.reset_index(inplace = True)
Device_2_Group_3HVA_Wide=Device_2_Group_3HVA.pivot_table(index=['deviceSegment','accountType'],columns=['Group Name'],values=['dfaActions'])

Device_2_Group_floodlightActivity=Device_2_Group_New
del Device_2_Group_floodlightActivity['Group Name']
Device_2_Group_53Activities=Device_2_Group_floodlightActivity.pivot_table(index=['deviceSegment','accountType'],columns=['floodlightActivity'],values=['dfaActions'])
Device_2_Group_3HVA_Wide.columns =Device_2_Group_3HVA_Wide.columns.get_level_values(1)
Device_2_Group_3HVA_Wide.reset_index(inplace=True)
Device_2_Group_53Activities.columns =Device_2_Group_53Activities.columns.get_level_values(1)
Device_2_Group_53Activities.reset_index(inplace=True)

Merge_1_Device=pd.merge(Device_1,Device_2_Group_3HVA_Wide,on=["deviceSegment","accountType"])
Merge_2_Device=pd.merge(Merge_1_Device,Device_2_Group_53Activities,on=["deviceSegment","accountType"])
Merge_2_Device['Conversions']=np.sum(Merge_2_Device.loc[:,['Perry Ellis - HVA 1','Perry Ellis - HVA 2','Perry Ellis - HVA 3']],axis=1)
Merge_2_Device['ROI']=Merge_2_Device['dfaRevenue']/Merge_2_Device['cost']
Merge_2_Device['Click Through Rate']=Merge_2_Device['clicks']/Merge_2_Device['impr']
Merge_2_Device['Cost per Transaction']=Merge_2_Device['cost']/Merge_2_Device['dfaTransactions']  
Merge_2_Device['Cost per Click']=Merge_2_Device['cost']/Merge_2_Device['clicks']
Merge_2_Device['Cost per HVA (total)']=Merge_2_Device['cost']/Merge_2_Device['Conversions']
Merge_2_Device['Cost per HVA 1']=Merge_2_Device['cost']/Merge_2_Device['Perry Ellis - HVA 1']
Merge_2_Device['Cost per HVA 2']=Merge_2_Device['cost']/Merge_2_Device['Perry Ellis - HVA 2']
Merge_2_Device['Cost per HVA 3']=Merge_2_Device['cost']/Merge_2_Device['Perry Ellis - HVA 3']

cols = Merge_2_Device.columns.tolist()
cols_new=cols[:2]+['ROI']+cols[2:7]+['Conversions']+cols[-7:]+cols[7:-9]
Merge_2_Device_Final=Merge_2_Device[cols_new]
Merge_2_Device_Final=Merge_2_Device_Final.sort_values(by=['accountType','ROI'],ascending=[True,False])

# Weekday

In [5]:
# Weekday
creds = create_credentials()

service = get_service(creds)


print(start_date), print(end_date)
REPORTID_nonHVA = request_report(service, start_date, end_date, 
                                 ['accountType', 'date', 'clicks', 'cost', 'impr','dfaRevenue', 'dfaTransactions'])
REPORTID_HVA = request_report(service, start_date, end_date, 
                              ['accountType', 'date',
                               'floodlightActivity', 'dfaActions'])
non_hva_date= poll_report(service, REPORTID_nonHVA)
hva_date= poll_report(service, REPORTID_HVA)

2017-07-18
2017-09-04
'Report is not ready. I will try again.'
'The report is ready.'
'Downloading fragment 0 for report AAAnr-tNbFrmxsfK'
'The report is ready.'
'Downloading fragment 0 for report AAAnr6TarOafa4Aw'


In [6]:
Weekday_1=non_hva_date
Weekday_2=hva_date
import calendar
Weekday_1['weekday']=Weekday_1['date'].apply(lambda x: calendar.day_name[pd.to_datetime(x).weekday()])
Weekday_2['weekday']=Weekday_2['date'].apply(lambda x: calendar.day_name[pd.to_datetime(x).weekday()])

Weekday_1_Agg=Weekday_1.groupby(['accountType','weekday'])[['clicks','cost','impr','dfaRevenue','dfaTransactions']].sum()
Weekday_1_Agg.reset_index(inplace=True)

Weekday_2_Agg=Weekday_2.groupby(['accountType','weekday','floodlightActivity'])[['dfaActions']].sum()
Weekday_2_Agg.reset_index(inplace=True)

Weekday_2_Group_New=pd.merge(Weekday_2_Agg,Group,on="floodlightActivity")


Weekday_2_Group_3HVA=Weekday_2_Group_New.groupby(['accountType','weekday','Group Name'])[['dfaActions']].sum() 
Weekday_2_Group_3HVA.reset_index(inplace = True)
Weekday_2_Group_3HVA_Wide=Weekday_2_Group_3HVA.pivot_table(index=['weekday','accountType'],columns=['Group Name'],values=['dfaActions'])

Weekday_2_Group_floodlightActivity=Weekday_2_Group_New
del Weekday_2_Group_floodlightActivity['Group Name']
Weekday_2_Group_53Activities=Weekday_2_Group_floodlightActivity.pivot_table(index=['weekday','accountType'],columns=['floodlightActivity'],values=['dfaActions'])
Weekday_2_Group_3HVA_Wide.columns =Weekday_2_Group_3HVA_Wide.columns.get_level_values(1)
Weekday_2_Group_3HVA_Wide.reset_index(inplace=True)
Weekday_2_Group_53Activities.columns =Weekday_2_Group_53Activities.columns.get_level_values(1)
Weekday_2_Group_53Activities.reset_index(inplace=True)

Merge_1_Weekday=pd.merge(Weekday_1_Agg,Weekday_2_Group_3HVA_Wide,on=["weekday","accountType"])
Merge_2_Weekday=pd.merge(Merge_1_Weekday,Weekday_2_Group_53Activities,on=["weekday","accountType"])
Merge_2_Weekday['Conversions']=np.sum(Merge_2_Weekday.loc[:,['Perry Ellis - HVA 1','Perry Ellis - HVA 2','Perry Ellis - HVA 3']],axis=1)
Merge_2_Weekday['ROI']=Merge_2_Weekday['dfaRevenue']/Merge_2_Weekday['cost']
Merge_2_Weekday['Click Through Rate']=Merge_2_Weekday['clicks']/Merge_2_Weekday['impr']
Merge_2_Weekday['Cost per Transaction']=Merge_2_Weekday['cost']/Merge_2_Weekday['dfaTransactions']  
Merge_2_Weekday['Cost per Click']=Merge_2_Weekday['cost']/Merge_2_Weekday['clicks']
Merge_2_Weekday['Cost per HVA (total)']=Merge_2_Weekday['cost']/Merge_2_Weekday['Conversions']
Merge_2_Weekday['Cost per HVA 1']=Merge_2_Weekday['cost']/Merge_2_Weekday['Perry Ellis - HVA 1']
Merge_2_Weekday['Cost per HVA 2']=Merge_2_Weekday['cost']/Merge_2_Weekday['Perry Ellis - HVA 2']
Merge_2_Weekday['Cost per HVA 3']=Merge_2_Weekday['cost']/Merge_2_Weekday['Perry Ellis - HVA 3']

cols = Merge_2_Weekday.columns.tolist()
cols_new=cols[:2]+['ROI']+cols[2:7]+['Conversions']+cols[-7:]+cols[7:-9]
Merge_2_Weekday_Final=Merge_2_Weekday[cols_new]

Merge_2_Weekday_Final=Merge_2_Weekday_Final.sort_values(by=['accountType','ROI'],ascending=[True,False])

# Hour

In [9]:
hour1=pd.read_excel("/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/Auto Run Device Weekday and Hour/Hour Data/Hour 1.xlsx")
hour2=pd.read_excel("/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/Auto Run Device Weekday and Hour/Hour Data/Hour 2.xlsx")
Hour_of_day_1=hour1[["Engine","Hour of day","Clicks","Cost","Impr","Revenue","Trans"]]
Hour_of_day_2=hour2[["Engine","Hour of day","Floodlight activity","Actions"]]

Group = pd.read_csv("/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/Auto Run Device Weekday and Hour/Group.csv")

Hour_of_day_2_Group_New=pd.merge(Hour_of_day_2,Group,  on="Floodlight activity")

Hour_of_day_2_Group_3HVA=Hour_of_day_2_Group_New.groupby(['Engine','Hour of day','Group Name'],as_index=False)['Actions'].sum()
Hour_of_day_2_Group_3HVA_Wide=Hour_of_day_2_Group_3HVA.pivot_table(index=['Hour of day','Engine'],columns=['Group Name'],values=['Actions'])
Hour_of_day_2_Group_3HVA_Wide.columns =Hour_of_day_2_Group_3HVA_Wide.columns.get_level_values(1)
Hour_of_day_2_Group_3HVA_Wide.reset_index(inplace=True)

Hour_of_day_2_Group_floodlightActivity=Hour_of_day_2_Group_New

Hour_of_day_2_Group_53Activities_Wide=Hour_of_day_2_Group_floodlightActivity.pivot_table(index=['Hour of day','Engine'],columns=['Floodlight activity'],values=['Actions'],dropna=False)
Hour_of_day_2_Group_53Activities_Wide.columns =Hour_of_day_2_Group_53Activities_Wide.columns.get_level_values(1)
Hour_of_day_2_Group_53Activities_Wide.reset_index(inplace=True)
##
Merge_1_Hour=pd.merge(Hour_of_day_1,Hour_of_day_2_Group_3HVA_Wide,on=["Hour of day","Engine"])
Merge_2_Hour_of_day=pd.merge(Merge_1_Hour,Hour_of_day_2_Group_53Activities_Wide,on=["Hour of day","Engine"])

Merge_2_Hour_of_day['Conversions']=np.sum(Merge_2_Hour_of_day.loc[:,['Perry Ellis - HVA 1','Perry Ellis - HVA 2','Perry Ellis - HVA 3']],axis=1)
Merge_2_Hour_of_day['ROI']=Merge_2_Hour_of_day['Revenue']/Merge_2_Hour_of_day['Cost']
Merge_2_Hour_of_day['Click Through Rate']=Merge_2_Hour_of_day['Clicks']/Merge_2_Hour_of_day['Impr']
Merge_2_Hour_of_day['Cost per Transaction']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Trans']  
Merge_2_Hour_of_day['Cost per Click']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Clicks']
Merge_2_Hour_of_day['Cost per HVA (total)']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Conversions']
Merge_2_Hour_of_day['Cost per HVA 1']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Perry Ellis - HVA 1']
Merge_2_Hour_of_day['Cost per HVA 2']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Perry Ellis - HVA 2']
Merge_2_Hour_of_day['Cost per HVA 3']=Merge_2_Hour_of_day['Cost']/Merge_2_Hour_of_day['Perry Ellis - HVA 3']

cols = Merge_2_Hour_of_day.columns.tolist()
cols_new=cols[:2]+['ROI']+cols[2:7]+['Conversions']+cols[-7:]+cols[7:-9]
Hour_of_day_Final=Merge_2_Hour_of_day[cols_new]

Hour_of_day_Final=Hour_of_day_Final.sort_values(by=['Engine','ROI'],ascending=[True,False])
# To change later #### Hour_of_day_Final.to_excel('Aug. 21/Weekday.xlsx', sheet_name='Hour_of_day_Final', index=False)



In [10]:
list_dfs=[Merge_2_Device_Final,Merge_2_Weekday_Final,Hour_of_day_Final]
def save_xls(list_dfs, xls_path):
    writer = ExcelWriter(xls_path)
    for n, df in enumerate(list_dfs):
        df.to_excel(writer,'sheet%s' % n,index=False)
    writer.save()
save_xls(list_dfs,xls_path)